# Openterface Hot-Plug Detection Demo

This notebook demonstrates the hot-plugging device detection functionality. It will:
1. Store initial device detection data
2. Monitor for device changes (connect/disconnect)
3. Compare newly detected data with the initial stored data
4. Display detailed information about device changes

In [ ]:
import sys
import os
import time
from datetime import datetime
import threading

# Add the parent directory to the path to import device modules
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), '..'))

# Import cross-platform device modules
from device import DeviceFactory
from device.AbstractDeviceManager import DeviceSelector
from utils import logger

# Configure logging
CoreLogger = logger.core_logger

print("✅ Cross-platform modules imported successfully!")
print(f"📁 Working directory: {os.getcwd()}")
print(f"🐍 Python path: {sys.path[0]}")
print(f"🖥️  Current platform: {DeviceFactory.get_current_platform()}")
print(f"✅ Platform supported: {DeviceFactory.is_platform_supported()}")

In [ ]:
# Device configuration - Openterface VID/PID values
HID_VID = "534D"
HID_PID = "2109"
Serial_port_VID = "1A86"
Serial_port_PID = "7523"

print(f"🔍 Device Configuration:")
print(f"   Serial Port - VID:{Serial_port_VID} PID:{Serial_port_PID}")
print(f"   HID Device  - VID:{HID_VID} PID:{HID_PID}")

# Create cross-platform device manager and selector
device_manager = DeviceFactory.create_device_manager(Serial_port_VID, Serial_port_PID, HID_VID, HID_PID)
device_selector = DeviceSelector(device_manager)

print(f"✅ Device manager created for {DeviceFactory.get_current_platform()} platform")

def format_device_brief(device):
    """Format device info briefly for display"""
    parts = []
    if device.get('serial_port_path'):
        parts.append(f"Serial:{device['serial_port_path']}")
    if device.get('camera_path'):
        parts.append(f"Video")
    if device.get('audio_path'):
        parts.append(f"Audio")
    if device.get('HID_path'):
        parts.append(f"HID")
    return " | ".join(parts) if parts else "Unknown device"

def device_change_callback(event_data):
    """Callback function called when device changes are detected"""
    print(f"\n🚨 === Device Change Detected at {event_data['timestamp']} ===")
    
    changes_from_last = event_data['changes_from_last']
    changes_from_initial = event_data['changes_from_initial']
    
    # Report changes from last scan
    if changes_from_last['added_devices']:
        print(f"📱 NEW DEVICES CONNECTED ({len(changes_from_last['added_devices'])}):")
        for device in changes_from_last['added_devices']:
            print(f"  ➕ {format_device_brief(device)}")
    
    if changes_from_last['removed_devices']:
        print(f"🔌 DEVICES DISCONNECTED ({len(changes_from_last['removed_devices'])}):")
        for device in changes_from_last['removed_devices']:
            print(f"  ➖ {format_device_brief(device)}")
    
    if changes_from_last['modified_devices']:
        print(f"🔄 DEVICES MODIFIED ({len(changes_from_last['modified_devices'])}):")
        for change in changes_from_last['modified_devices']:
            print(f"  🔀 {format_device_brief(change['new'])}")
    
    # Report overall state compared to initial
    current_count = len(event_data['current_devices'])
    initial_count = len(event_data['initial_snapshot'].devices)
    print(f"📊 Total devices: {current_count} (was {initial_count} initially)")
    
    if changes_from_initial['added_devices']:
        print(f"📈 Net new since start: {len(changes_from_initial['added_devices'])}")
    
    if changes_from_initial['removed_devices']:
        print(f"📉 Net removed since start: {len(changes_from_initial['removed_devices'])}")
    
    print("=" * 60)

print("✅ Configuration and callback function defined!")

In [ ]:
# Create and configure the cross-platform hotplug monitor
print("🔧 Creating Cross-Platform Hotplug Monitor...")

monitor = DeviceFactory.create_hotplug_monitor(
    serial_vid=Serial_port_VID,
    serial_pid=Serial_port_PID,
    hid_vid=HID_VID,
    hid_pid=HID_PID,
    poll_interval=2.0  # Check every 2 seconds
)

# Add our callback function
monitor.add_callback(device_change_callback)

print("✅ Cross-platform hotplug monitor created and configured!")
print(f"⏰ Polling interval: {monitor.poll_interval} seconds")
print(f"🖥️  Platform: {DeviceFactory.get_current_platform()}")

In [ ]:
# Start monitoring and capture initial device state
print("🚀 Starting hotplug monitoring...")

# Start the monitoring process
monitor.start_monitoring()

# Wait a moment for initial scan to complete
time.sleep(1)

# Display initial device state
initial_state = monitor.get_initial_state()
if initial_state:
    print(f"\n📋 Initial Device State (captured at {initial_state['timestamp']}):")
    print(f"   Found {initial_state['device_count']} device(s)")
    for i, device in enumerate(initial_state['devices'], 1):
        print(f"   {i}. {format_device_brief(device)}")
        print(f"      Serial Port: {device.get('serial_port_path', 'N/A')}")
        print(f"      HID Path: {device.get('HID_path', 'N/A')}")
        print(f"      Camera: {device.get('camera_path', 'N/A')}")
        print(f"      Audio: {device.get('audio_path', 'N/A')}")
        print()
else:
    print("⚠️  No initial devices found matching the specified VID/PID")

print("🟢 Monitoring is now active!")
print("💡 Connect or disconnect Openterface devices to see changes in real-time")

In [ ]:
# Check current monitoring status and device selection by port chain
# Run this cell to see the current device state at any time

current_state = monitor.get_current_state()
initial_state = monitor.get_initial_state()

if current_state and initial_state:
    print(f"📊 Current Monitoring Status:")
    print(f"   Initial devices: {initial_state['device_count']} (at {initial_state['timestamp']})")
    print(f"   Current devices: {current_state['device_count']} (at {current_state['timestamp']})")
    
    # Compare current with initial
    current_snapshot = monitor.device_manager.create_snapshot()
    changes = current_snapshot.compare_with(monitor.initial_snapshot)
    
    if changes['added_devices'] or changes['removed_devices']:
        print(f"\n🔄 Changes since start:")
        if changes['added_devices']:
            print(f"   ➕ Added: {len(changes['added_devices'])} devices")
        if changes['removed_devices']:
            print(f"   ➖ Removed: {len(changes['removed_devices'])} devices")
    else:
        print(f"\n✅ No changes since monitoring started")
        
    print(f"\n🏃‍♂️ Monitor is {'running' if monitor.running else 'stopped'}")

    # Show available port chains for device selection
    print(f"\n🔗 Available Port Chains:")
    port_chains = device_selector.device_manager.list_available_port_chains()
    for i, port_chain in enumerate(port_chains, 1):
        devices = device_selector.device_manager.get_devices_by_port_chain(port_chain)
        print(f"   {i}. {port_chain} ({len(devices)} device(s))")
        for device in devices:
            print(f"      - {device}")
else:
    print("⚠️  Monitor not initialized or no data available")

In [ ]:
# Display detailed device information and device selection
# Run this cell to see detailed information about all currently detected devices

def display_detailed_device_info(devices, title="Device Information"):
    print(f"📱 {title}")
    print("=" * 60)
    
    if not devices:
        print("   No devices found")
        return
    
    for i, device in enumerate(devices, 1):
        print(f"Device {i}:")
        print(f"   Port Chain: {device.get('port_chain', 'Unknown')}")
        print(f"   Serial Port ID: {device.get('serial_port', 'Not found')}")
        print(f"   Serial Port Path: {device.get('serial_port_path', 'Not found')}")
        print(f"   HID ID: {device.get('HID', 'Not found')}")
        print(f"   HID Path: {device.get('HID_path', 'Not found')}")
        print(f"   Camera ID: {device.get('camera', 'Not found')}")
        print(f"   Camera Path: {device.get('camera_path', 'Not found')}")
        print(f"   Audio ID: {device.get('audio', 'Not found')}")
        print(f"   Audio Path: {device.get('audio_path', 'Not found')}")
        print()

# Display initial devices
if monitor.initial_snapshot:
    initial_devices = [dev.to_dict() if hasattr(dev, 'to_dict') else dev for dev in monitor.initial_snapshot.devices]
    display_detailed_device_info(initial_devices, "Initial Devices")

# Display current devices
current_devices = monitor.device_manager.discover_devices()
current_devices_dict = [dev.to_dict() for dev in current_devices]
display_detailed_device_info(current_devices_dict, "Current Devices")

# Show device selection by port chain
print("🎯 Device Selection by Port Chain:")
print("=" * 60)

grouped_devices = device_selector.list_devices_grouped_by_port_chain()
if grouped_devices:
    for port_chain, devices in grouped_devices.items():
        print(f"Port Chain: {port_chain}")
        for device in devices:
            print(f"  - {device}")
        print()
        
    # Example: Select first available device
    if grouped_devices:
        first_port_chain = list(grouped_devices.keys())[0]
        selected_device = device_selector.select_device_by_port_chain(first_port_chain)
        if selected_device:
            print(f"🎯 Example - Selected device from port chain '{first_port_chain}':")
            print(f"   {selected_device}")
else:
    print("   No devices available for selection")

In [ ]:
# Stop monitoring
# Run this cell when you want to stop the hotplug monitoring

print("🛑 Stopping cross-platform hotplug monitoring...")
monitor.stop_monitoring()
print("✅ Monitoring stopped successfully!")

# Display final summary
final_state = monitor.get_current_state()
initial_state = monitor.get_initial_state()

if final_state and initial_state:
    print(f"\n📈 Final Summary:")
    print(f"   Platform: {DeviceFactory.get_current_platform()}")
    print(f"   Monitoring duration: {final_state['timestamp'] - initial_state['timestamp']}")
    print(f"   Initial device count: {initial_state['device_count']}")
    print(f"   Final device count: {final_state['device_count']}")
    
    # Calculate final changes
    final_snapshot = monitor.device_manager.create_snapshot()
    final_changes = final_snapshot.compare_with(monitor.initial_snapshot)
    
    if final_changes['added_devices'] or final_changes['removed_devices']:
        print(f"   Net changes:")
        if final_changes['added_devices']:
            print(f"     ➕ Added: {len(final_changes['added_devices'])} devices")
        if final_changes['removed_devices']:
            print(f"     ➖ Removed: {len(final_changes['removed_devices'])} devices")
    else:
        print(f"   ✅ No net changes detected")
        
print("\n🎯 Cross-platform hotplug detection demo completed!")

## Cross-Platform Device Hot-Plug Detection Demo

### How to Use This Demo

1. **Run cells 1-4 in order** to set up and start cross-platform monitoring
2. **Connect or disconnect Openterface devices** - you'll see real-time notifications
3. **Run cell 5** anytime to check current status and available port chains
4. **Run cell 6** to see detailed device information and device selection examples
5. **Run cell 7** when you want to stop monitoring

### Cross-Platform Features

✅ **Abstract device management** - Platform-independent interface  
✅ **Device selection by port chain** - Select devices using unique port identifiers  
✅ **Initial device snapshot** - Captures baseline device state  
✅ **Real-time monitoring** - Detects device connect/disconnect events  
✅ **Change comparison** - Compares current state with initial state  
✅ **Detailed logging** - Shows comprehensive device information  
✅ **Event callbacks** - Custom functions triggered on device changes  

### Supported Platforms

- ✅ **Windows** - Fully implemented using Windows APIs
- 🔄 **Linux** - TODO: Implementation needed
- 🔄 **macOS** - TODO: Implementation needed

### Device Selection by Port Chain

The port chain provides a unique identifier for each physical USB port/hub configuration:
- **Format**: `"2-1.2"` (Hub-Port.Interface)
- **Usage**: Users can select specific devices by their port chain
- **Benefit**: Consistent device identification across reconnections

### Device Information Tracked

- **Serial Port**: Communication interface device
- **HID Device**: Human Interface Device 
- **Camera**: Video capture device
- **Audio**: Audio input/output device
- **Port Chain**: Physical USB topology identifier

Each device includes both the device ID and the actual system path for access.

### API Usage Examples

```python
# Create device manager for current platform
device_manager = DeviceFactory.create_device_manager("1A86", "7523", "534D", "2109")

# Create device selector for port-based selection
device_selector = DeviceSelector(device_manager)

# List available port chains
port_chains = device_manager.list_available_port_chains()

# Select device by port chain
device = device_selector.select_device_by_port_chain("2-1.2")

# Create hotplug monitor
monitor = DeviceFactory.create_hotplug_monitor("1A86", "7523", "534D", "2109")
```